#Install

In [ ]:
!pip install easyocr
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Imports

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
import cv2
import easyocr

#Globals

In [ ]:
BATCH_SIZE = 4
LR = 1e-4
NUM_CLASSES = 2  # 1: plate, 0: background
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

TRAIN_IMG_DIR = "/content/drive/MyDrive/archive/train/train"
TRAIN_ANNO_PATH = "/content/drive/MyDrive/archive/train/train/_annotations.csv"
VALID_IMG_DIR = "/content/drive/MyDrive/archive/valid/valid"
VALID_ANNO_PATH = "/content/drive/MyDrive/archive/valid/valid/_annotations.csv"

MODEL_PATH = "plate_detection.pth"  # Model kayıt/okuma için


#Utils

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = int(max(widthA, widthB))
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = int(max(heightA, heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def get_plate_corners(box, image_shape, margin=0.08):
    xmin, ymin, xmax, ymax = box
    h, w = image_shape[:2]
    dx = (xmax - xmin) * margin
    dy = (ymax - ymin) * margin
    xmin = max(0, xmin - dx)
    xmax = min(w - 1, xmax + dx)
    ymin = max(0, ymin - dy)
    ymax = min(h - 1, ymax + dy)
    pts = np.array([
        [xmin, ymin], [xmax, ymin],
        [xmax, ymax], [xmin, ymax]
    ], dtype="float32")
    return pts


#Data

In [ ]:
class PlateDetectionDataset(Dataset):
    def __init__(self, image_dir, csv_file, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms
        self.df = pd.read_csv(csv_file)
        self.df = self.df[self.df['class'] == 'License_Plate'].reset_index(drop=True)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        boxes = [[row['xmin'], row['ymin'], row['xmax'], row['ymax']]]
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((1,), dtype=torch.int64)
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx]),
        }
        if self.transforms:
            img = self.transforms(img)
        return img, target

    def __len__(self):
        return len(self.df)

# Dataloaders:
train_dataset = PlateDetectionDataset(TRAIN_IMG_DIR, TRAIN_ANNO_PATH, transforms=get_transform(train=True))
valid_dataset = PlateDetectionDataset(VALID_IMG_DIR, VALID_ANNO_PATH, transforms=get_transform(train=False))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


#Network

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn

def get_detection_model(num_classes=2):
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes
    )
    return model

model = get_detection_model(NUM_CLASSES)
model.to(DEVICE)
# Eğer önceden eğitilmiş model varsa:
# model.load_state_dict(torch.load(MODEL_PATH))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 215MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

#Train

In [ ]:
optimizer = torch.optim.SGD([p for p in model.parameters() if p.requires_grad],
                           lr=LR, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0
    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        total_loss += losses.item()
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    lr_scheduler.step()
    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

# Eğitim döngüsü
num_epochs = 25
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, DEVICE, epoch)
    print(f"Epoch {epoch+1} completed.")
# torch.save(model.state_dict(), MODEL_PATH)


Epoch 1 | Loss: 294.7401
Epoch 1 completed.
Epoch 2 | Loss: 215.3766
Epoch 2 completed.
Epoch 3 | Loss: 205.5681
Epoch 3 completed.
Epoch 4 | Loss: 200.7332
Epoch 4 completed.
Epoch 5 | Loss: 198.7519
Epoch 5 completed.
Epoch 6 | Loss: 199.4526
Epoch 6 completed.
Epoch 7 | Loss: 199.7124
Epoch 7 completed.
Epoch 8 | Loss: 198.3747
Epoch 8 completed.
Epoch 9 | Loss: 199.7764
Epoch 9 completed.
Epoch 10 | Loss: 199.1331
Epoch 10 completed.


#Evaluation

In [ ]:
reader = easyocr.Reader(['en'])

def detect_and_reconstruct_plate(image_path, detection_model, device, conf_thresh=0.7):
    orig_img = Image.open(image_path).convert("RGB")
    np_img = np.array(orig_img)
    transform = T.ToTensor()
    img = transform(orig_img).to(device)
    detection_model.eval()
    with torch.no_grad():
        preds = detection_model([img])
    boxes = preds[0]['boxes']
    scores = preds[0]['scores']
    results = []
    for i, score in enumerate(scores):
        if score > conf_thresh:
            box = boxes[i].cpu().numpy()
            pts = get_plate_corners(box, np_img.shape)
            warped_plate = four_point_transform(np_img, pts)
            ocr_result = reader.readtext(warped_plate, detail=0)
            results.append((box, warped_plate, ocr_result))
    return results

# Test bir görüntüde dene:
test_img_path = "/content/drive/MyDrive/archive/valid/valid/00071c51c8e92a68_jpg.rf.446f345e7d5974082616d32dd8eb3d27.jpg"
results = detect_and_reconstruct_plate(test_img_path, model, DEVICE)

for i, (box, plate_img, ocr_text) in enumerate(results):
    print(f"Plate {i+1} BBox: {box}")
    print(f"OCR: {ocr_text}")
    cv2.imwrite(f"plate_{i+1}_reconstructed.jpg", plate_img)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompletePlate 1 BBox: [525.3186  481.38364 647.7286  554.74835]
OCR: ['CRoss']
